In [ ]:
import os
import psycopg2
import google.cloud.bigquery as bigquery
import pandas as pd


PGHOST = 'musicbrainz.postgres.database.azure.com'
PGDATABASE = os.environ.get('PGDATABASE', 'musicbrainz')
PGUSER = os.environ.get('PGUSER', 'musicbrainz')
PGPASSWORD = os.environ.get('PGPASSWORD', 'Git62036242')
cnx = psycopg2.connect(host=PGHOST,database=PGDATABASE, 
                      user=PGUSER, password=PGPASSWORD)
crs = cnx.cursor()


In [ ]:
import sqldf
pollstar_data = pd.read_csv("E:\\GeorgiaTech\\cse6242\\CS6242_Fall_2023_JJBASK\\data\\pollstar_data.csv", encoding='utf-8')
pollstar_data = pollstar_data.drop('Unnamed: 0', axis=1)
pollstar_data.columns = ['artist', 'gross', 'tickets_sold']
pollstar_data['artist'] = pollstar_data['artist'].str.removesuffix("\r")
pollstar_data['artist'] = pollstar_data['artist'].str.strip() 
pollstar_data = pollstar_data.dropna()
print(f"pollstar_data recs : {pollstar_data.shape[0]}")
all_records = []
for index, row in pollstar_data.iterrows():
    #print(f"artist from pollstar {row['artist']}")
    id_value = row['artist']
    sql_query = "SELECT *  FROM musicbrainz.artist as mb_artist WHERE mb_artist.name LIKE  %s"        
    crs.execute(sql_query, (id_value,))
    records = crs.fetchall()
    #print(f"record {records}")    
    for record in records:
        all_records.append(record)      

column_names = [desc[0] for desc in crs.description if crs.description]
df_A = pd.DataFrame(all_records, columns=column_names)
print(f"df_A recs : {df_A.shape[0]}")

df_A.head()


In [ ]:
from psycopg2.extras import execute_values
query = """
SELECT a.id, a.gid, a.name, a.type, a.area, a.gender, psd.gross, psd.tickets_sold
FROM pollstar_data as psd JOIN df_A as a ON  a.name like psd.artist
"""

df_B = sqldf.run(query)
df_B.head()
print(f"df_C recs : {df_B.shape[0]}")
df_B = df_B.rename(columns={'gross': 'gross_revenue'})
df_B['gross_revenue'] = df_B['gross_revenue'].str.replace('[\$,]', '', regex=True).astype('int64')
df_B['tickets_sold'] = df_B['tickets_sold'].str.replace('[\,]', '', regex=True).astype('int64')
df_B['type'] = df_B['type'].fillna(0).astype('int')
df_B['area'] = df_B['area'].fillna(0).astype('int')
df_B['gender'] = df_B['gender'].fillna(0).astype('int')
df_B.head()
insert_query = 'INSERT INTO pollstar_top150_artist (id, gid,name,type,area,gender,gross_revenue,tickets_sold) VALUES %s'
tuples = [tuple(x) for x in df_B.to_numpy()]
execute_values(crs, insert_query, tuples)
cnx.commit()

In [ ]:
cnx.close()